
# CSD 실습 문제 답안


In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels as sm
import matplotlib
import matplotlib.pyplot as plt
import warnings
import time

In [2]:
import CSD

## 1. 계산 실행

'block_transfer_list_Raw_Data.csv' 예제 파일을 분석하고 결과를 출력해보자.

#### 입력 조건

1. 분석 대상 공정 시작 날짜, 끝 날짜 칼럼 이름 :
SD = ['ASSY_ACTL_SD','OFT_ACTL_SD','PNT_ACTL_SD']
FD = ['ASSY_ACTL_FD','OFT_ACTL_FD','PNT_ACTL_FD']

2. 공정 이름 :
process_names = ['ASSY_ACTL','OFT_ACTL','PNT_ACTL']

3. 기계 대수 : 310대, 275대, 250대

4. 분석 기간 : 2012년 1월 1일 - 2016년 1월 1일



In [3]:
csv_name = 'block_transfer_list_Raw_Data.csv'
SD = ['ASSY_ACTL_SD','OFT_ACTL_SD','PNT_ACTL_SD']
FD = ['ASSY_ACTL_FD','OFT_ACTL_FD','PNT_ACTL_FD']
process_names = ['ASSY_ACTL','OFT_ACTL','PNT_ACTL']
set_m = [310, 275, 250]

In [4]:
temp = time.time()
csd = CSD.CSD_Calculator(csv_name, start_dates = SD, final_dates = FD, set_m = set_m, set_u = None,
                 process_names = process_names, time_range = [20120101, 20160101], del_nan = True, del_inconsistency = True)
print(time.time() - temp)

2.6124401092529297


## 2. 결과 출력

#### 출력 대상

1. var_table
2. result_table
3. df_dates
4. df_te
5. iat

### 1) var_table

In [5]:
csd.var_table

,ASSY_ACTL,OFT_ACTL,PNT_ACTL
Ra,11.9712,11.9712,11.9712
Ca,3.47665,7.30993,6.90088
Te,17.189,15.9965,12.838
Ce,1.26909,1.47839,0.658405
Rd,8.28951,9.61784,10.4712
Cd,37.7186,12.7912,6.72059
m,310,275,250
u,0.663786,0.696355,0.614745


### 2) result_table

In [6]:
csd.result_table

,ASSY_ACTL,OFT_ACTL,PNT_ACTL
CTq,6.1981e-05,0.000262473,9.33449e-06
CT,17.1891,15.9968,12.838
WIPq,0.000741987,0.00314212,0.000111745
WIP,205.774,191.501,153.686


### 3) df_dates

In [7]:
csd.df_dates.head()

,ASSY_ACTL_SD_S,OFT_ACTL_SD_S,PNT_ACTL_SD_S,ASSY_ACTL_FD_F,OFT_ACTL_FD_F,PNT_ACTL_FD_F
8040,2012-02-14,2012-01-30,2012-05-18,2012-03-14,2012-02-08,2012-06-08
8175,2012-01-04,2012-02-20,2012-02-23,2012-02-23,2012-02-20,2012-02-27
8176,2012-01-05,2012-02-13,2012-02-15,2012-02-09,2012-02-14,2012-02-20
8177,2012-01-02,2012-02-20,2012-02-22,2012-02-21,2012-02-21,2012-02-27
8180,2012-02-09,2012-02-15,2012-02-22,2012-02-14,2012-02-17,2012-02-29


In [8]:
csd.df_dates.describe()

,ASSY_ACTL_SD_S,OFT_ACTL_SD_S,PNT_ACTL_SD_S,ASSY_ACTL_FD_F,OFT_ACTL_FD_F,PNT_ACTL_FD_F
count,17467,17467,17467,17467,17467,17467
unique,1409,1389,1168,1112,1335,1235
top,2015-11-20 00:00:00,2015-12-14 00:00:00,2015-01-11 00:00:00,2015-08-31 00:00:00,2015-11-30 00:00:00,2015-11-30 00:00:00
freq,55,59,50,101,76,68
first,2012-01-02 00:00:00,2011-12-09 00:00:00,2011-12-23 00:00:00,2012-01-06 00:00:00,2012-01-07 00:00:00,2012-01-19 00:00:00
last,2015-12-31 00:00:00,2016-07-12 00:00:00,2016-07-26 00:00:00,2017-10-13 00:00:00,2016-12-27 00:00:00,2016-08-13 00:00:00


### 4) df_te

In [9]:
csd.df_te.head()

,ASSY_ACTL,OFT_ACTL,PNT_ACTL
8040,30.0,10.0,22.0
8175,51.0,1.0,5.0
8176,36.0,2.0,6.0
8177,51.0,2.0,6.0
8180,6.0,3.0,8.0


### 5) df_iat

In [10]:
csd.iat

array([[  0.,   7.,   0.,   0.,   2.,   2.],
       [  0.,   5.,   5.,   0.,   0.,   5.],
       [  0.,   5.,  16.,   0.,   1.,   0.],
       ...,
       [  0.,  61.,   9.,   0.,   1.,  18.],
       [  0.,   6.,   1.,  17.,  56.,   3.],
       [  0.,  50.,  55., 599., 158.,  58.]])

## ## 원하는 utilization 값 설정

위 1번 예제와 모두 같은 설정으로 하되, 원하는 utilization 수준을 95%로 맞추도록 해보자.

※ 주의 : 같은 파일, 같은 칼럼을 분석하더라도 새로운 CSD_Calculator 클래스를 생성할 때는 csv_name, start_dates, final_dates를 아래 셀과 같이 다시 지정해 준 후에 생성해야 한다.
※ 주의2 : set_u 값은 0과 1 사이 숫자로 해주도록 한다.

In [11]:
csv_name = 'block_transfer_list_Raw_Data.csv'
SD = ['ASSY_ACTL_SD','OFT_ACTL_SD','PNT_ACTL_SD']
FD = ['ASSY_ACTL_FD','OFT_ACTL_FD','PNT_ACTL_FD']
process_names = ['ASSY_ACTL','OFT_ACTL','PNT_ACTL']
set_u = 0.95

In [12]:
### 원하는 utilization을 맞춰주는 기능
temp = time.time()
csd = CSD.CSD_Calculator(csv_name, start_dates = SD, final_dates = FD, set_m = None, set_u = set_u,
                 process_names = process_names, time_range = [20120101, 20160101], del_nan = True, del_inconsistency = True)
print(time.time() - temp)

2.5874249935150146


utilization이 0.9로 잘 맞춰졌는지 확인하기 위해 var_table을 호출해보자.

In [13]:
csd.var_table

,ASSY_ACTL,OFT_ACTL,PNT_ACTL
Ra,11.9712,11.9712,11.9712
Ca,3.47665,7.30993,6.90088
Te,17.189,15.9965,12.838
Ce,1.26909,1.47839,0.658405
Rd,8.28951,9.61784,10.4712
Cd,37.7186,12.7912,6.72059
m,217,202,162
u,0.948266,0.948008,0.94868


## 3. 분포 함수 피팅

각 공정의 작업시간, 첫 번째 공정의 도착간 시간에 대해 분포함수를 피팅하고 각 분포에 대해 100만 개의 난수를 생성해서 확인해보자.

#### 입력 조건

1. 각 공정 작업시간 : 튜닝 사용, quantile = 0.99, accuracy = 0.01, 반올림 사용
2. 첫번째 공정(ASSY_ACTL) 도착간 시간(iat) : 튜닝 사용, quantile = None, accuracy = 0.0001, 반올림 사용


### 1-1) 각 공정 작업시간 피팅

In [14]:
dist_list_te, params_list_te = CSD.distribution_finder(csd.df_te, tunning = True, quantile = 0.99, accuracy = 0.01, _round = True)

Distribution 0 is tunning...
Tunned error : 0.005551857762180035
Distribution 1 is tunning...
Tunned error : 0.0004050610545789368
Distribution 2 is tunning...
Tunned error : 0.008056143316357733



Found distributions are :
0 : exponnorm(K=6.96, loc=4.92, scale=1.55)
1 : gamma(a=0.73, loc=1.00, scale=18.34)
2 : exponnorm(K=1.76, loc=7.65, scale=2.63)


Comparing original data with fitted distribution
0 : 
           Original        Fitting
count  17293.000000  999973.000000
mean      15.721159      15.707263
std       12.416968      10.898811
min        1.000000       0.000000
25%        8.000000       8.000000
50%       12.000000      13.000000
75%       19.000000      20.000000
max       87.000000     171.000000

CV
Original : 0.79    Fitting : 0.69



1 : 
           Original         Fitting
count  17296.000000  1000000.000000
mean      14.354186       14.340680
std       14.926468       15.611064
min        1.000000        1.000000
25%        4.000000        4.000000
50%       10.

### 1-2) 피팅한 분포함수로 난수 생성

In [15]:
random = []

for dist, params in zip(dist_list_te, params_list_te) :

    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    temp = dist.rvs(arg, loc, scale, size = 1000000)
    temp = temp[temp > 0] # 처리 옵션 : nonnegative
    temp = np.round(temp) # 처리 옵션 : 반올림

    temp = pd.Series(temp)
    random.append(temp)


for rand in random:
    print("CV : " + str(round(rand.std() / rand.mean(),2)))
    print(rand.describe())
    print("\n\n")

CV : 0.7
count    999974.000000
mean         15.722087
std          10.927112
min           0.000000
25%           8.000000
50%          13.000000
75%          20.000000
max         195.000000
dtype: float64



CV : 1.09
count    1000000.000000
mean          14.364770
std           15.668585
min            1.000000
25%            4.000000
50%            9.000000
75%           19.000000
max          239.000000
dtype: float64



CV : 0.43
count    999773.000000
mean         12.289943
std           5.325152
min           0.000000
25%           9.000000
50%          11.000000
75%          15.000000
max          71.000000
dtype: float64





### 2-1) 첫 번째 공정 도착간 시간 피팅

In [16]:
dist_list_iat, params_list_iat = CSD.distribution_finder(csd.iat[:,1], tunning = True, quantile = None, accuracy = 0.0001, _round = True)

Distribution 0 is tunning...
Tunned error : 4.211508072826664e-05



Found distributions are :
0 : chi2(df=1.53, loc=-0.00, scale=0.13)


Comparing original data with fitted distribution
0 : 
           Original         Fitting
count  17466.000000  1000000.000000
mean       0.096015        0.096731
std        0.701884        0.301351
min        0.000000        0.000000
25%        0.000000        0.000000
50%        0.000000        0.000000
75%        0.000000        0.000000
max       61.000000        4.000000

CV
Original : 7.31    Fitting : 3.12



Process finished in 11.038495779037476 sec


In [17]:
dist = dist_list_iat[0] #위의 distribution_finder 함수에서 반환한 분포함수들의 리스트에서 원하는 함수 선택
params = params_list_iat[0] #위의 distribution_finder 함수에서 반환한 파라미터들의 리스트에서 원하는 파라미터 선택

#이 부분은 그냥 따라하면 된다
arg = params[:-2]
loc = params[-2]
scale = params[-1]
#선택한 분포에 따른 난수 생성
a = dist.rvs(arg, loc, scale, size = 1000000)
a = a[a > 0] # 처리 옵션 : nonnegative
a = np.round(a) # 처리 옵션 : 반올림

#데이터 타입을 pd.Series로 변환하여 난수가 제대로 발생됐는지 확인
a = pd.Series(a)
print("CV : " + str(round(a.std() / a.mean(),2)))
a.describe()

CV : 3.1


count    1000000.000000
mean           0.097287
std            0.302007
min            0.000000
25%            0.000000
50%            0.000000
75%            0.000000
max            4.000000
dtype: float64